In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time

In [2]:
# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


17464789/17464789 [==============================] - 6s 0us/step


In [3]:
# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [4]:
# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

1641221/1641221 [==============================] - 0s 0us/step


In [5]:
# Create and fit tokenizer
#tokenizer = Tokenizer(num_words=max_features)
#tokenizer.fit_on_texts(X_train)


In [6]:
# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
# Train the model
batch_size = 32
epochs = 5
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

In [9]:
start_time = time.time()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
training_time = time.time() - start_time


Epoch 1/5


782/782 [==============================] - 128s 156ms/step - loss: 0.4186 - accuracy: 0.8040 - val_loss: 0.3392 - val_accuracy: 0.8497
Epoch 2/5
782/782 [==============================] - 117s 150ms/step - loss: 0.2953 - accuracy: 0.8759 - val_loss: 0.3517 - val_accuracy: 0.8440
Epoch 3/5
782/782 [==============================] - 113s 145ms/step - loss: 0.2506 - accuracy: 0.8988 - val_loss: 0.3563 - val_accuracy: 0.8482
Epoch 4/5
782/782 [==============================] - 116s 148ms/step - loss: 0.2090 - accuracy: 0.9175 - val_loss: 0.3962 - val_accuracy: 0.8369
Epoch 5/5
782/782 [==============================] - 117s 149ms/step - loss: 0.1794 - accuracy: 0.9295 - val_loss: 0.4120 - val_accuracy: 0.8419


In [10]:
# Evaluate the model
start_time = time.time()

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
prediction_time = time.time() - start_time

print("Test accuracy:", acc)

782/782 [==============================] - 18s 23ms/step - loss: 0.4120 - accuracy: 0.8419
Test accuracy: 0.841920018196106


In [11]:
#print("Test accuracy:", acc)
#print("Training time:", training_time, "seconds")
#print("Prediction time:", prediction_time, "seconds")

In [12]:
# Function to preprocess user input
def preprocess_input(user_input):
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)
   
#from tensorflow.keras.preprocessing.text import Tokenizer
    #tokenizer = Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(user_input)
    #sequences = tokenizer.texts_to_sequences(user_input)
    #return pad_sequences(sequences, maxlen=maxlen)

In [13]:
# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [14]:
# User Interface
print("Welcome to Movie Review Sentiment Analyzer!")

Welcome to Movie Review Sentiment Analyzer!


In [16]:
while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)

Enter your movie review (or type 'quit' to exit): good movie
1/1 [==============================] - 0s 72ms/step
Predicted sentiment: Positive
Enter your movie review (or type 'quit' to exit): You can tell its directed by a South Indian but that's what make it beautiful. Insane cameos, powerful cinematography. Imo one of the best SRK action movies. The fights were engaging, little humor everywhere created very good atmosphere.  After watching Pathaan where they had a very dull climax, I wasn't hyped a lot for this movie, hell barely saw the trailer but entering the cinema my views changed. Flying start, story was somewhat new but also had a lot of inspiration from previous series like Money Heist.  Someone who never hates on South Indian style action, this movie provides just that but with a high budget which you can see throughout the movie.
1/1 [==============================] - 0s 61ms/step
Predicted sentiment: Positive
Enter your movie review (or type 'quit' to exit): I just could 